## Configurações Pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

25/07/16 15:55:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/16 15:55:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/16 15:55:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Configurações Pandas

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [3]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched')
           )

### Condition Occurrence

In [4]:
window_condition_occurrence = Window.partitionBy().orderBy('person_id', 'dtnasc_sinasc')

In [ ]:
# /data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/cte_vocab_map_standard.csv

In [26]:
df_condition_occurrence = (df_input
                           .withColumn('cod_id_agravo', F.substring(F.col('`id_agravo.y`'), 2,2))
                           .withColumn('cod_id_agravo_letra', F.substring(F.col('`id_agravo.y`'), 1,1))
                           .withColumn('person_id', F.col('person_id'))
                           # criação do campo condition_concept_id
                           .withColumn('id_agravo_cat', F.when((F.col('cod_id_agravo')>=15)&(F.col('cod_id_agravo')<=19), 4145952).otherwise(0))
                           # O raiox_tora tem os seguintes valores: 0,1,2,3,4,99,null
                           # Danilo está vendo se é só isso ou precisa adicionar para os outros casos, além do 2
                           .withColumn('raiox_tora_cat', F.when((F.col('id_agravo_cat')==4145952)&(F.col('raiox_tora')==1), 36674325)
                                                          .when((F.col('id_agravo_cat')==4145952)&(F.col('raiox_tora')==2), 45768800)
                                                          .when((F.col('id_agravo_cat')==4145952)&(F.col('raiox_tora')==3), 257907) # pode mudar
                                                          .when((F.col('id_agravo_cat')==4145952)&(F.col('raiox_tora')==4), 4118638)
                                                          .otherwise(0))
                           .withColumn('concept_id', F.concat_ws(",", F.col('id_agravo_cat'), F.col('raiox_tora_cat'))) 
                           .withColumn('condition_concept_id', F.explode(F.split(F.col('concept_id'), ',')))
                           .withColumn('condition_source_concept_id', F.when((F.col('condition_concept_id').isin(4145952))&(F.col('forma')==0), F.lit(4080894))
                                                                       .when((F.col('condition_concept_id').isin(4145952))&(F.col('forma')==1), F.lit(253954))
                                                                       .when((F.col('condition_concept_id').isin(4145952))&(F.col('forma')==2), F.lit(4310580))
                                                                       .when((F.col('condition_concept_id').isin(4145952))&(F.col('forma')==3), F.lit(4013013))
                                                                       .when((F.col('condition_concept_id').isin(4145952))&(F.col('forma')==99), F.lit(4127662))
                                                                       .otherwise(None))
                           .withColumn('condition_type_concept_id', F.lit(32879))
                           .withColumn('condition_start_datetime', F.lit(None))
                           .withColumn('condition_end_date', F.lit(None))
                           .withColumn('condition_end_datetime', F.lit(None))
                           .withColumn('stop_reason', F.when(F.col('raiox_tora')==0, None)
                                                       .when(F.col('raiox_tora')==1, 4090826)
                                                       .when(F.col('raiox_tora')==2, 4056683)
                                                       .when(F.col('raiox_tora')==3, 4049904)
                                                       .when(F.col('raiox_tora')==4, 4118638)
                                                       .when(F.col('raiox_tora')==4, 4127662)
                                                       .otherwise(None))
                           .withColumn('condition_status_concept_id', F.lit(None))
                           .withColumn('provider_id', F.lit(None))
                           .withColumn('visit_occurrence_id', F.col('visit_occurrence_id'))
                           .withColumn('visit_detail_id', F.lit(None))
                           .withColumn('condition_source_value', F.when(F.col('condition_concept_id').isin(4145952, 45768800), F.col('`id_agravo.y`')).otherwise(0))
                           .withColumn('condition_status_source_value', F.lit(None))
                           # A coluna condition_start_date não pode ser nula, então onde for null vou colocar temp. o dtnasc_sinasc:
                           .withColumn('condition_start_date', F.to_date(F.coalesce(F.col('dt_noti_at'), F.col('dtnasc_sinasc')), 'yyyy-MM-dd'))
                           .withColumn('condition_occurrence_id', F.row_number().over(window_condition_occurrence))
                                ).select(
                                    F.col('condition_occurrence_id').cast('integer'),
                                    F.col('person_id').cast('integer'),
                                    F.col('condition_concept_id').cast('integer'),
                                    F.col('condition_start_date').cast('date'),
                                    F.col('condition_start_datetime').cast('timestamp'),
                                    F.col('condition_end_date').cast('date'),
                                    F.col('condition_end_datetime').cast('timestamp'),
                                    F.col('condition_type_concept_id').cast('integer'),
                                    F.col('condition_status_concept_id').cast('integer'),
                                    F.col('stop_reason').cast('string'),
                                    F.col('provider_id').cast('integer'),
                                    F.col('visit_occurrence_id').cast('integer'),
                                    F.col('visit_detail_id').cast('integer'),
                                    F.col('condition_source_value').cast('string'),
                                    F.col('condition_source_concept_id').cast('integer'),
                                    F.col('condition_status_source_value').cast('string')
                                        )

df_condition_occurrence.count()

33181108

## Salvando dados no Postgresql

In [27]:
write_to_postgresl(df_condition_occurrence, tb_name='tuberculose.condition_occurrence', write_mode='append')

25/07/16 16:31:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 16:31:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 16:31:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 16:31:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 16:31:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


'Total time: 0:29:14.061505 and - Total rows: 33181108 - Total columns: 16'

## Gerando Vocab map

-- CTE_VOCAB_MAP_SOURCE
CREATE TABLE tuberculose.CTE_VOCAB_MAP_SOURCE as 
WITH CTE_VOCAB_MAP_SOURCE_ AS (
       SELECT 
	   		c.concept_code AS SOURCE_CODE, 
			c.concept_id AS SOURCE_CONCEPT_ID, 
			c.CONCEPT_NAME AS SOURCE_CODE_DESCRIPTION, 
			c.vocabulary_id AS SOURCE_VOCABULARY_ID, 
			c.domain_id AS SOURCE_DOMAIN_ID, 
			c.concept_class_id AS SOURCE_CONCEPT_CLASS_ID, 
            c.VALID_START_DATE AS SOURCE_VALID_START_DATE, 
			c.VALID_END_DATE AS SOURCE_VALID_END_DATE, 
			c.invalid_reason AS SOURCE_INVALID_REASON, 
            c.concept_ID as TARGET_CONCEPT_ID, 
			c.concept_name AS TARGET_CONCEPT_NAME, 
			c.vocabulary_id AS TARGET_VOCABULARY_ID, 
			c.domain_id AS TARGET_DOMAIN_ID, 
			c.concept_class_id AS TARGET_CONCEPT_CLASS_ID, 
			c.INVALID_REASON AS TARGET_INVALID_REASON, 
            c.STANDARD_CONCEPT AS TARGET_STANDARD_CONCEPT
       FROM CONCEPT c
       UNION
       SELECT 
	   		source_code, SOURCE_CONCEPT_ID, 
			SOURCE_CODE_DESCRIPTION, 
			source_vocabulary_id, 
			c1.domain_id AS SOURCE_DOMAIN_ID, 
			c2.CONCEPT_CLASS_ID AS SOURCE_CONCEPT_CLASS_ID, 
			c1.VALID_START_DATE AS SOURCE_VALID_START_DATE, 
			c1.VALID_END_DATE AS SOURCE_VALID_END_DATE,
			stcm.INVALID_REASON AS SOURCE_INVALID_REASON,
			target_concept_id, 
			c2.CONCEPT_NAME AS TARGET_CONCEPT_NAME, 
			target_vocabulary_id, 
			c2.domain_id AS TARGET_DOMAIN_ID, 
			c2.concept_class_id AS TARGET_CONCEPT_CLASS_ID, 
			c2.INVALID_REASON AS TARGET_INVALID_REASON, 
			c2.standard_concept AS TARGET_STANDARD_CONCEPT
       FROM source_to_concept_map stcm
              LEFT OUTER JOIN CONCEPT c1
                     ON c1.concept_id = stcm.source_concept_id
              LEFT OUTER JOIN CONCEPT c2
                     ON c2.CONCEPT_ID = stcm.target_concept_id
       WHERE stcm.INVALID_REASON IS NULL
)
SELECT *
FROM CTE_VOCAB_MAP_SOURCE_


-- CTE_VOCAB_MAP_STANDARD
CREATE TABLE tuberculose.CTE_VOCAB_MAP_STANDARD as 
WITH CTE_VOCAB_MAP AS (
       SELECT 
			c.concept_code AS SOURCE_CODE, 
			c.concept_id AS SOURCE_CONCEPT_ID, 
			c.concept_name AS SOURCE_CODE_DESCRIPTION, 
			c.vocabulary_id AS SOURCE_VOCABULARY_ID, 
			c.domain_id AS SOURCE_DOMAIN_ID, 
			c.CONCEPT_CLASS_ID AS SOURCE_CONCEPT_CLASS_ID, 
			c.VALID_START_DATE AS SOURCE_VALID_START_DATE, 
			c.VALID_END_DATE AS SOURCE_VALID_END_DATE, 
			c.INVALID_REASON AS SOURCE_INVALID_REASON, 
			c1.concept_id AS TARGET_CONCEPT_ID, 
			c1.concept_name AS TARGET_CONCEPT_NAME, 
			c1.VOCABULARY_ID AS TARGET_VOCABULARY_ID, 
			c1.domain_id AS TARGET_DOMAIN_ID, 
			c1.concept_class_id AS TARGET_CONCEPT_CLASS_ID, 
			c1.INVALID_REASON AS TARGET_INVALID_REASON, 
			c1.standard_concept AS TARGET_STANDARD_CONCEPT
       FROM CONCEPT C
             JOIN CONCEPT_RELATIONSHIP CR
                        ON C.CONCEPT_ID = CR.CONCEPT_ID_1
                        AND CR.invalid_reason IS NULL
                        AND lower(cr.relationship_id) = 'maps to'
              JOIN CONCEPT C1
                        ON CR.CONCEPT_ID_2 = C1.CONCEPT_ID
                        AND C1.INVALID_REASON IS NULL
       UNION ALL
       SELECT 
			source_code, 
			SOURCE_CONCEPT_ID, 
			SOURCE_CODE_DESCRIPTION, 
			source_vocabulary_id, 
			c1.domain_id AS SOURCE_DOMAIN_ID,
			c2.CONCEPT_CLASS_ID AS SOURCE_CONCEPT_CLASS_ID,
			c1.VALID_START_DATE AS SOURCE_VALID_START_DATE, 
			c1.VALID_END_DATE AS SOURCE_VALID_END_DATE, 
			stcm.INVALID_REASON AS SOURCE_INVALID_REASON,
			target_concept_id, 
			c2.CONCEPT_NAME AS TARGET_CONCEPT_NAME, 
			target_vocabulary_id, 
			c2.domain_id AS TARGET_DOMAIN_ID, 
			c2.concept_class_id AS TARGET_CONCEPT_CLASS_ID, 
			c2.INVALID_REASON AS TARGET_INVALID_REASON, 
			c2.standard_concept AS TARGET_STANDARD_CONCEPT
       FROM source_to_concept_map stcm
              LEFT OUTER JOIN CONCEPT c1
                     ON c1.concept_id = stcm.source_concept_id
              LEFT OUTER JOIN CONCEPT c2
                     ON c2.CONCEPT_ID = stcm.target_concept_id
       WHERE stcm.INVALID_REASON IS NULL
)
SELECT *
FROM CTE_VOCAB_MAP